In [1]:
import os
import pickle
# import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from scipy.stats import norm

# from utils_subdivision.gen_distribution_single_plots import analyze_phases
# from utils_subdivision.gen_distribution_subplot import analyze_single_type    # plot_combined_results
# from utils_subdivision.gen_distribution_merged_plot import plot_merged
# from utils_dot_plot.drum_single import analyze_phases
# from utils_dot_plot.drum_merged import plot_merged_per_mode

# from utils_subdivision.gen_distribution_subplot import analyze_single_type
from utils_dot_plot.kinematic_dot_plot import *
from utils_dot_plot.drum_merged import *
from utils_dot_plot.drum_dance_piece import *


PIECE_TYPES = ["Suku", "Maraka", "Manjanin", "Wasulunka"]

base_output_dir =  "output_dot_plots"
by_piece_dir = os.path.join(base_output_dir, "by_piece")
os.makedirs(base_output_dir, exist_ok=True)
os.makedirs(by_piece_dir, exist_ok=True)


# Merged Drum Dot Plot Per Piece

In [ ]:
# # # Main execution code --------------------------------------------------------------------
# m_idx = 2
# mode = ["group", "individual", "audience"]
# dance_mode = mode[m_idx]

# # Dictionary to store drum_phases_kde for each piece
# piece_drum_phases_kde = {}

# with open('data/selected_piece_list.pkl', 'rb') as f:
#     piece_list = pickle.load(f)

# # Collect data for all pieces
# for file_name in piece_list:
#     print(file_name)
    
#     # Determine piece type
#     PIECE_TYPES = ["Suku", "Maraka", "Manjanin", "Wasulunka"]
#     piece_type = next((p for p in PIECE_TYPES if p in file_name), None)
#     if piece_type is None:
#         print(f"Warning: Unknown piece type for {file_name}")
#         continue
    
#     # Load data paths
#     cycles_csv_path = f"data/virtual_cycles/{file_name}_C.csv"
#     onsets_csv_path = f"data/drum_onsets/{file_name}.csv"
#     dmode_path = f"data/dance_modes_ts/{file_name}_{dance_mode}.pkl"
    
#     if os.path.exists(dmode_path):
#         with open(dmode_path, "rb") as f:
#             dance_mode_time_segments = pickle.load(f)
#     else:
#         continue
    
#     # Get drum phases and KDE data
#     _, _, drum_phases_kde = plot_merged_stacked(
#         file_name=file_name,
#         dance_mode=dance_mode,
#         cycles_csv_path=cycles_csv_path,
#         onsets_csv_path=onsets_csv_path,
#         dance_mode_time_segments=dance_mode_time_segments,
#         figsize=(10, 3),
#         dpi=200,
#         use_window=True
#     )
    
#     # Store the data
#     if piece_type not in piece_drum_phases_kde:
#         piece_drum_phases_kde[piece_type] = []
#     piece_drum_phases_kde[piece_type].append(drum_phases_kde)

# save_path = os.path.join(base_output_dir, f"piece_drum_phases_kde_{dance_mode}.pkl")
# with open(save_path, 'wb') as f:
#     pickle.dump(piece_drum_phases_kde, f)
# print(f"\nSaved piece KDE data to: {save_path}")

In [4]:

m_idx = 2
mode = ["group", "individual", "audience"]
dance_mode = mode[m_idx]
load_path1 = os.path.join(base_output_dir, f"piece_drum_phases_kde_{dance_mode}.pkl")


with open(load_path1, 'rb') as f:        
    piece_drum_phases_kde = pickle.load(f)      # This was saved in the previous cell
    
    
PIECE_TYPES = ["Suku", "Maraka", "Manjanin", "Wasulunka"]
# Create combined plots for each piece type
for piece_type in PIECE_TYPES:
    if piece_type in piece_drum_phases_kde:
        fig, ax = plot_combined_merged_stacked(
            piece_type=piece_type,
            dance_mode=dance_mode,
            drum_phases_kde_all=piece_drum_phases_kde[piece_type],
            legend_flag=False
        )
        
        
        # plt.show()
        # Save the figure
        save_dir1 = os.path.join(by_piece_dir, "drum_kde_by_piece", dance_mode)
        os.makedirs(save_dir1, exist_ok=True)
        
        save_path = os.path.join(save_dir1, f"{piece_type}_{dance_mode}_combined.png")
        plt.savefig(save_path, bbox_inches='tight')
        plt.close()

# Generate Dance dot plot per piece

In [ ]:
# # Main execution code --------------------------------------------------------------------
# m_idx = 0
# mode = ["group", "individual", "audience"]
# dance_mode = mode[m_idx]

# # Dictionary to store dance_phases_kde for each piece type
# piece_dance_phases_kde = {}

# with open('data/selected_piece_list.pkl', 'rb') as f:
#     piece_list = pickle.load(f)

# # Collect data for all pieces
# for file_name in piece_list:
#     print(file_name)
    
#     # Determine piece type
#     piece_type = next((p for p in PIECE_TYPES if p in file_name), None)
#     if piece_type is None:
#         print(f"Warning: Unknown piece type for {file_name}")
#         continue
    
#     # Load data paths
#     cycles_csv_path = f"data/virtual_cycles/{file_name}_C.csv"
#     left_onset_path = f"data/logs_v4_0.007_foot_jun3/{file_name}_T/onset_info/{file_name}_T_left_foot_onsets.csv"
#     right_onset_path = f"data/logs_v4_0.007_foot_jun3/{file_name}_T/onset_info/{file_name}_T_right_foot_onsets.csv"
#     dmode_path = f"data/dance_modes_ts/{file_name}_{dance_mode}.pkl"
    
#     left_onsets = pd.read_csv(left_onset_path)["time_sec"].values
#     right_onsets = pd.read_csv(right_onset_path)["time_sec"].values
    
#     if os.path.exists(dmode_path):
#         with open(dmode_path, "rb") as f:
#             dance_mode_time_segments = pickle.load(f)
#     else:
#         continue
    
#     # Get foot phases and KDE data
#     fig, ax, dance_phases_kde = plot_foot_onsets_stacked(
#         file_name=file_name,
#         dance_mode=dance_mode,
#         cycles_csv_path=cycles_csv_path,
#         left_onsets=left_onsets,
#         right_onsets=right_onsets,
#         dance_mode_time_segments=dance_mode_time_segments,
#         figsize=(10, 3),
#         dpi=200,
#         use_window=True
#     )
    
#     # Store the data
#     if piece_type not in piece_dance_phases_kde:
#         piece_dance_phases_kde[piece_type] = []
#     piece_dance_phases_kde[piece_type].append(dance_phases_kde)
    
#     # Close the individual piece figure
#     plt.close(fig)


# save_path = os.path.join(base_output_dir, f"piece_dance_phases_kde_{dance_mode}.pkl")
# with open(save_path, 'wb') as f:
#     pickle.dump(piece_dance_phases_kde, f)
# print(f"\nSaved piece KDE data to: {save_path}")

In [7]:
m_idx = 2
mode = ["group", "individual", "audience"]
dance_mode = mode[m_idx]


load_path2 = os.path.join(base_output_dir, f"piece_dance_phases_kde_{dance_mode}.pkl")

with open(load_path2, 'rb') as f:
    piece_dance_phases_kde = pickle.load(f)      # This was saved in the previous cell

# Create combined plots for each piece type
for piece_type in PIECE_TYPES:
    if piece_type in piece_dance_phases_kde:
        fig, ax = plot_combined_foot_stacked(
            piece_type=piece_type,
            dance_mode=dance_mode,
            dance_phases_kde_all=piece_dance_phases_kde[piece_type],
            legend_flag=False
        )
        
        # Save the figure
        save_dir2 = os.path.join(by_piece_dir, "dance_kde_by_piece", dance_mode)
        os.makedirs(save_dir2, exist_ok=True)
        
        save_path = os.path.join(save_dir2, f"{piece_type}_{dance_mode}_combined.png")
        plt.savefig(save_path, bbox_inches='tight')
        plt.close()

# Combined drum-dance kde plot

In [10]:
# Main execution code
m_idx = 0
mode = ["group", "individual", "audience"]
dance_mode = mode[m_idx]

PIECE_TYPES = ["Suku", "Maraka", "Manjanin", "Wasulunka"]

# Load the saved data
load_drum_path = os.path.join(base_output_dir, f"piece_drum_phases_kde_{dance_mode}.pkl")
load_dance_path = os.path.join(base_output_dir, f"piece_dance_phases_kde_{dance_mode}.pkl")


with open(load_drum_path, 'rb') as f:        
    piece_drum_phases_kde = pickle.load(f) 

with open(load_dance_path, 'rb') as f:
    piece_dance_phases_kde = pickle.load(f)

# Create combined plots for each piece type
for piece_type in PIECE_TYPES:
    if piece_type in piece_drum_phases_kde and piece_type in piece_dance_phases_kde:
        plot_combined_drum_dance(
            piece_type=piece_type,
            dance_mode=dance_mode,
            drum_phases_kde_all=piece_drum_phases_kde[piece_type],
            dance_phases_kde_all=piece_dance_phases_kde[piece_type],
            save_dir = by_piece_dir,
            legend_flag=False
        )
        
